In [ ]:
import socket

def start_server(host='0.0.0.0', port=12345):
    # Create a TCP/IP socket
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    # Bind the socket to the address and port
    server_socket.bind((host, port))
    print(f"Server bound to {host}:{port}")
    
    # Listen for incoming connections (the parameter is the backlog)
    server_socket.listen(1)
    print("Server is listening for incoming connections...")
    
    # Accept a connection
    client_socket, client_address = server_socket.accept()
    print(f"Connected to by {client_address}")
    
    # Send a greeting message to the client
    client_socket.sendall(b"Hello from server")
    
    # Receive a response from the client
    data = client_socket.recv(1024)
    print("Received from client:", data.decode())
    
    # Close the connection
    client_socket.close()
    server_socket.close()

if __name__ == '__main__':
    while True: start_server()

In [1]:
!pip install gspread google-auth


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import gspread
from google.oauth2.service_account import Credentials


In [15]:

# Define the scope and authenticate using your service account credentials
SCOPE = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

CREDS = Credentials.from_service_account_file("remote-control-452220-bd30dfe8afd5.json", scopes=SCOPE)
CLIENT = gspread.authorize(CREDS)


In [18]:

# Open the spreadsheet by name and select the first sheet
SHEET = CLIENT.open_by_key("1knBUxNbgVLqWpQUln39AFZ5k-kuV32KsNZ1bWw0Qiww").sheet1


In [19]:

def register_client(client_id, ip, port):
    """
    Registers or updates the client in the Google Sheet.
    The sheet is assumed to have a header row with columns: client_id, ip, port.
    """
    records = SHEET.get_all_records()
    row_index = None

    # Look for an existing record with the same client_id
    for i, record in enumerate(records):
        if record.get("client_id") == client_id:
            row_index = i + 2  # +2 because get_all_records() skips header and sheet rows are 1-indexed
            break

    if row_index:
        # Update the existing record
        SHEET.update_cell(row_index, 2, ip)
        SHEET.update_cell(row_index, 3, port)
        print(f"Updated registration for {client_id}")
    else:
        # Append a new record
        SHEET.append_row([client_id, ip, port])
        print(f"Registered new client {client_id}")


In [20]:

def lookup_client(target_id):
    """
    Looks up the target client's connection details.
    Returns a tuple (ip, port) or (None, None) if not found.
    """
    records = SHEET.get_all_records()
    for record in records:
        if record.get("client_id") == target_id:
            return record.get("ip"), record.get("port")
    return None, None


In [25]:

# Example usage:
if __name__ == "__main__":
    # For registration, detect your public IP (via an external service or NAT traversal)
    my_client_id = "client1"
    my_public_ip = "203.0.113.45"  # Replace with your actual public IP
    my_listen_port = "6000"
    register_client(my_client_id, my_public_ip, my_listen_port)

    # To connect to another client (say "client2"), look up its connection info
    target_id = "client1"
    target_ip, target_port = lookup_client(target_id)
    if target_ip and target_port:
        print(f"Found target {target_id} at {target_ip}:{target_port}")
        # You can now attempt a direct connection using these details
    else:
        print(f"Client {target_id} not found.")


Registered new client client1
Client client1 not found.


In [26]:
import requests

def get_public_ip():
    try:
        response = requests.get("https://api.ipify.org?format=json")
        response.raise_for_status()  # Raise an error for bad responses
        ip_data = response.json()
        return ip_data.get("ip")
    except requests.RequestException as e:
        print("Error fetching public IP:", e)
        return None

if __name__ == "__main__":
    public_ip = get_public_ip()
    if public_ip:
        print("Your public IP address is:", public_ip)


Your public IP address is: 154.198.86.255


Your public IP address is: 154.198.86.255
